Instalando as Bibliotecas que vamos utilizar

In [1]:
pip install pandas

In [2]:
pip install pandera

     |████████████████████████████████| 186 kB 37.7 MB/s 
     |████████████████████████████████| 161 kB 62.0 MB/s 


Importando as Bibliotecas com um apelido para facilitar o código

In [3]:
import pandas as pd
import pandera as pa
import numpy as np
import collections
from operator import itemgetter


Lendo o Arquivo ** CSV ** gerado do tratamento do arquivo      ** **"amazon_prime_titles.json"** **

---



In [23]:
amazon_df = pd.read_csv("/content/drive/MyDrive/ProjetoFinal/Amazon2021_tratado2.csv",dayfirst=True)

In [30]:
amazon_df.head(40)
drop. unnamed:0

,Unnamed: 0,id,tipo,titulo,diretor,elenco,pais,data_adicionado,ano_lancamento,classificacao,duracao,genero
0,0,s1,Filme,A Grande Sedução,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,2021-03-30,2014,Não avaliado,113 min,"Comedia, Drama"
1,1,s2,Filme,Cuide da boa noite,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,2021-03-30,2018,13+,110 min,"Drama, Internacional"
2,2,s3,Filme,Segredos da Decepção,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",Estados Unidos,2021-03-30,2017,Não avaliado,74 min,"Acao, Drama, Suspense"
3,3,s4,Filme,Rosa: Mantendo-se Verdadeiro,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",Estados Unidos,2021-03-30,2014,Não avaliado,69 min,Documentario
4,4,s5,Filme,Criador de Monstros,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",Reino Unido,2021-03-30,1989,Não avaliado,45 min,"Drama, Fantasia"
5,5,s6,Filme,Vivendo com Dinossauros,Paul Weiland,"Gregory Chisholm, Juliet Stevenson, Brian Hens...",Reino Unido,2021-03-30,1989,Não avaliado,52 min,"Fantasia, Infantil"
6,6,s7,Filme,Arma alugada,Fran Strine,"Alice Cooper, Liberty DeVitto, Ray Parker Jr.,...",Estados Unidos,2021-03-30,2017,Não avaliado,98 min,"Documentary, Special Interest"
7,7,s8,Filme,Grease Live,"Thomas Kail, Alex Rudzinski","Julianne Hough, Aaron Tveit, Vanessa Hudgens, ...",Estados Unidos,2021-03-30,2016,Não avaliado,131 min,Comedia
8,8,s9,Filme,Global Meltdown,Daniel Gilboy,"Michael Paré, Leanne Khol Young, Patrick J. Ma...",Canada,2021-03-30,2017,Não avaliado,87 min,"Action, Science Fiction, Suspense"
9,9,s10,Filme,Mãe de David,Robert Allan Ackerman,"Kirstie Alley, Sam Waterston, Stockard Channing",Estados Unidos,2021-04-01,1994,Não avaliado,92 min,Drama


Ainda encontramos inconsistencias  e continuamos tratando os dados
# **Tradução**

Alteração do campo data do formato "mes 24,2021" para "dd/mm/aa"
as linhas que contém contém informação vazia receberam "Não informado"

In [6]:
amazon_df.dtypes
amazon_df['data_adicionado'] = pd.to_datetime(amazon_df.data_adicionado)
amazon_df.dtypes

Unnamed: 0                  int64
id                         object
tipo                       object
titulo                     object
diretor                    object
elenco                     object
pais                       object
data_adicionado    datetime64[ns]
ano_lancamento              int64
classificacao              object
duracao                    object
genero                     object
dtype: object

In [7]:
df = amazon_df[['data_adicionado']].groupby(amazon_df.classificacao)
df.head(10)

,data_adicionado
0,2021-03-30
1,2021-03-30
2,2021-03-30
3,2021-03-30
4,2021-03-30
...,...
805,NaT
863,NaT
1126,NaT
1139,NaT


Em uma segunda análise abaixo mostra que das 9668 linhas do arquivo a coluna data_adicionada recebe vazio(nenhuma informação em 9513 linhas o que sugere que a coluna não seja relevante e por isso poderia ser dropada mas para não influenciar possíveis análises em conjunto com o Dataset Netflix será mantida

In [8]:
amazon_df.shape

(9668, 12)

In [9]:
na_df = pd.DataFrame(amazon_df.isna().sum(), columns=['Contagem de NaN'])
na_df.index.name = 'data_adicionado'
na_df[ na_df['Contagem de NaN']>0].sort_values('Contagem de NaN', ascending=False)

,Contagem de NaN
data_adicionado,
data_adicionado,9513
diretor,2082
elenco,1233


A coluna classificacao continha dados no padrão e linguagem americanos(USA) Foi feita uma pesquisa de equivalencia e traduzido para o padrão e idioma PT-BR

restou ainda a inconsistência de vazios que serão agora substituidos por "Não Avaliado"

In [10]:
pd.unique(amazon_df.classificacao)

array(['Não avaliado', '13+', 'Livre', '18+', 'Restrito', 'Infantil',
       '7+', 'Adultos', '16+', '14+', '16'], dtype=object)

In [11]:
#todos os campos desta coluna que estavam vazios recebem agora "Não avaliado"
amazon_df.classificacao.fillna("Não avaliado", inplace=True)

In [12]:
pd.unique(amazon_df.classificacao)

array(['Não avaliado', '13+', 'Livre', '18+', 'Restrito', 'Infantil',
       '7+', 'Adultos', '16+', '14+', '16'], dtype=object)

A coluna titulo dos filmes recebeu a tradução de alguns filmes. Por se tratar de uma lista extensa deve ser traduzida aos poucos porém não vimos a necessidades de concluir isso neste momento de análises

In [27]:
#Tradução dos dados da coluna titulo
amazon_df.loc[amazon_df.titulo=='Living With Dinosaurs',['titulo']]='Vivendo com Dinossauros'
amazon_df.loc[amazon_df.titulo=='Hired Gun',['titulo']]='Arma alugada'
amazon_df.loc[amazon_df.titulo=='Grease Live!',['titulo']]='Grease Live'
amazon_df.loc[amazon_df.titulo=='Global Meltdown',['titulo']]='Global Meltdown'
amazon_df.loc[amazon_df.titulo=="David's Mother",['titulo']]="Mãe de David"
amazon_df.loc[amazon_df.titulo=='Forest Fairies',['titulo']]='Forest Fairies'
amazon_df.loc[amazon_df.titulo=='Take Care',['titulo']]='Se cuide'
amazon_df.loc[amazon_df.titulo=='The Night Eats The World',['titulo']]='A noite come o mundo'
amazon_df.loc[amazon_df.titulo=='Resilencia',['titulo']]='Resiliência'
amazon_df.loc[amazon_df.titulo=='Elon Musk: The Real Life Iron Man',['titulo']]='Elon Musk: o homem de ferro da vida real'
amazon_df.loc[amazon_df.titulo=='Summer 03',['titulo']]='Verão 03'
amazon_df.loc[amazon_df.titulo=='Zoombies',['titulo']]='Zoombies'
amazon_df.loc[amazon_df.titulo=='Zoo Babies',['titulo']]='Bebês do zoológico'
amazon_df.loc[amazon_df.titulo=='Zoë Coombs Marr: Bossy Bottom',['titulo']]='Zoë Coombs Marr: fundo mandão'
amazon_df.loc[amazon_df.titulo=='Zo Zo Zombie: Mini-Series',['titulo']]='Zo Zo Zombie: minisséries'
amazon_df.loc[amazon_df.titulo=='Zis Boom Bah',['titulo']]='Zis Boom Bah'

amazon_df.loc[amazon_df.titulo=='Zandalee',['titulo']]='Zandalee'
amazon_df.loc[amazon_df.titulo=='Zambezi: Force of Life',['titulo']]='Zambeze: Força da Vida'
amazon_df.loc[amazon_df.titulo=='Zahara: The Return',['titulo']]='Zahara: o retorno'
amazon_df.loc[amazon_df.titulo=='Z: The Beginning of Everything',['titulo']]='Z: O começo de tudo'

amazon_df.head(20)

,Unnamed: 0,id,tipo,titulo,diretor,elenco,pais,data_adicionado,ano_lancamento,classificacao,duracao,genero
0,0,s1,Filme,A Grande Sedução,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,2021-03-30,2014,Não avaliado,113 min,"Comedia, Drama"
1,1,s2,Filme,Cuide da boa noite,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,2021-03-30,2018,13+,110 min,"Drama, Internacional"
2,2,s3,Filme,Segredos da Decepção,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",Estados Unidos,2021-03-30,2017,Não avaliado,74 min,"Acao, Drama, Suspense"
3,3,s4,Filme,Rosa: Mantendo-se Verdadeiro,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",Estados Unidos,2021-03-30,2014,Não avaliado,69 min,Documentario
4,4,s5,Filme,Criador de Monstros,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",Reino Unido,2021-03-30,1989,Não avaliado,45 min,"Drama, Fantasia"
5,5,s6,Filme,Vivendo com Dinossauros,Paul Weiland,"Gregory Chisholm, Juliet Stevenson, Brian Hens...",Reino Unido,2021-03-30,1989,Não avaliado,52 min,"Fantasia, Infantil"
6,6,s7,Filme,Arma alugada,Fran Strine,"Alice Cooper, Liberty DeVitto, Ray Parker Jr.,...",Estados Unidos,2021-03-30,2017,Não avaliado,98 min,"Documentary, Special Interest"
7,7,s8,Filme,Grease Live,"Thomas Kail, Alex Rudzinski","Julianne Hough, Aaron Tveit, Vanessa Hudgens, ...",Estados Unidos,2021-03-30,2016,Não avaliado,131 min,Comedia
8,8,s9,Filme,Global Meltdown,Daniel Gilboy,"Michael Paré, Leanne Khol Young, Patrick J. Ma...",Canada,2021-03-30,2017,Não avaliado,87 min,"Action, Science Fiction, Suspense"
9,9,s10,Filme,Mãe de David,Robert Allan Ackerman,"Kirstie Alley, Sam Waterston, Stockard Channing",Estados Unidos,2021-04-01,1994,Não avaliado,92 min,Drama


Tradução de todos os campos de Gêneros dos filmes

In [14]:
pd.unique(amazon_df.genero)

array(['Comedia, Drama', 'Drama, Internacional', 'Acao, Drama, Suspense',
       'Documentario', 'Drama, Fantasia', 'Fantasia, Infantil',
       'Documentary, Special Interest', 'Comedia',
       'Action, Science Fiction, Suspense', 'Drama', 'Aventura, Infantil',
       'Terror, Suspense', 'Documentario, Esporte',
       'Terror, Ficcao Cientifica', 'Kids, Special Interest',
       'Comedy, Talk Show and Variety', 'Science Fiction', 'Infantil',
       'Action, Anime, Comedy',
       'Arts, Entertainment, and Culture, Comedy, Drama', 'TV Shows',
       'Animacao, Anime, Fantasia', 'Action, Adventure, Animation',
       'Anime, Infantil', 'Music Videos and Concerts', 'Drama, Suspense',
       'Fitness, Special Interest',
       'Faith and Spirituality, Special Interest', 'Special Interest',
       'Fitness', 'Action, Military and War, Western',
       'Arts, Entertainment, and Culture, Comedy, Talk Show and Variety',
       'Action, Western', 'Documentary, Science Fiction',
       'Adven

In [17]:
def translate_genero(row:str)->str:
        row_cleaned = row.strip(',').strip(' ').split(',')
        genero_idx = np.where(np.isin(genero_en, row_cleaned))
        genero_idx_list = list(genero_idx)[0].tolist()
        genero_to_pt = str(itemgetter(*genero_idx_list)(genero_pt))
        return genero_to_pt

Tradução linha a linha 

In [31]:

amazon_df.loc[amazon_df.genero=='Documentary',['genero']]='Documentario'
amazon_df.loc[amazon_df.genero=='Drama, Fantasy',['genero']]='Drama, Fantasia'
amazon_df.loc[amazon_df.genero=='Drama, International',['genero']]='Drama, Internacional'
amazon_df.loc[amazon_df.genero=='Comedy, Drama',['genero']]='Comedia, Drama'
amazon_df.loc[amazon_df.genero=='Action, Drama, Suspense',['genero']]='Acao, Drama, Suspense'
amazon_df.loc[amazon_df.genero=='Fantasy, Kids',['genero']]='Fantasia, Infantil'
amazon_df.loc[amazon_df.genero=='Comedy, Drama, Science Fiction',['genero']]='Comedia, Drama, Ficcao Cientifica'
amazon_df.loc[amazon_df.genero=='Comedy, Horror',['genero']]='Comedia, Terror'
amazon_df.loc[amazon_df.genero=='Comedy',['genero']]='Comedia'
amazon_df.loc[amazon_df.genero=='Action, Animation',['genero']]='Acao, Animacao'
amazon_df.loc[amazon_df.genero=='Comedy, Drama, Science Fiction',['genero']]='Comedia, Drama, Ficcao Cientifica'
amazon_df.loc[amazon_df.genero=='Adventure, Kids',['genero']]='Aventura, Infantil'
amazon_df.loc[amazon_df.genero=='Horror, Suspense',['genero']]='Terror, Suspense'
amazon_df.loc[amazon_df.genero=='Action, Animation',['genero']]='Acao, Animacao'
amazon_df.loc[amazon_df.genero=='Science Fiction, Special Interest',['genero']]='Ficcao Cientifica'
amazon_df.loc[amazon_df.genero=='Talk Show',['genero']]='Entrevista'
amazon_df.loc[amazon_df.genero=='Action, Animation',['genero']]='Acao, Animacao'

amazon_df.loc[amazon_df.genero=='Sports',['genero']]='Esporte'
amazon_df.loc[amazon_df.genero=='Kids',['genero']]='Infantil'
amazon_df.loc[amazon_df.genero=='Entertainment',['genero']]='Entretenimento'
amazon_df.loc[amazon_df.genero=='Animation, Anime, Fantasy',['genero']]='Animacao, Anime, Fantasia'
amazon_df.loc[amazon_df.genero=='Anime, Kids',['genero']]='Anime, Infantil'
amazon_df.loc[amazon_df.genero=='Animation',['genero']]='Animacao'
amazon_df.loc[amazon_df.genero=='Variety',['genero']]='Variedade'
amazon_df.loc[amazon_df.genero=='Action, Drama',['genero']]='Acao, Drama'

amazon_df.loc[amazon_df.genero=='Sports',['genero']]='Esporte'
amazon_df.loc[amazon_df.genero=='Kids',['genero']]='Infantil'
amazon_df.loc[amazon_df.genero=='Action, Adventure, Comedy',['genero']]='Acao, Aventura Comedia'
amazon_df.loc[amazon_df.genero=='Arts, Entertainment, and Culture, Documentary',['genero']]='Arte, Entretenimento, Cultura, Documentario'
amazon_df.loc[amazon_df.genero=='Documentary, LGBTQ',['genero']]='Documentario, LGBTQ'
amazon_df.loc[amazon_df.genero=='Culture',['genero']]='Cultura'
amazon_df.loc[amazon_df.genero=='Horror, Science Fiction',['genero']]='Terror, Ficcao Cientifica'
amazon_df.loc[amazon_df.genero=='Documentary, Sports',['genero']]='Documentario, Esporte'
amazon_df.loc[amazon_df.genero=='Drama, Fantasy',['genero']]='Drama, Fantasia'
amazon_df.loc[amazon_df.genero=='Drama, International',['genero']]='Drama, Internacional'
amazon_df.loc[amazon_df.genero=='Documentary',['genero']]='Documentario'
amazon_df.loc[amazon_df.genero=='Comedy, Drama',['genero']]='Comedia, Drama'
amazon_df.loc[amazon_df.genero=='Action, Drama, Suspense',['genero']]='Acao, Drama, Suspense'
amazon_df.loc[amazon_df.genero=='Fantasy, Kids',['genero']]='Fantasia, Infantil'
amazon_df.loc[amazon_df.genero=='Comedy, Drama, Science Fiction',['genero']]='Comedia, Drama, Ficcao Cientifica'
amazon_df.loc[amazon_df.genero=='Comedy, Horror',['genero']]='Comedia, Terror'
amazon_df.loc[amazon_df.genero=='Comedy',['genero']]='Comedia'
amazon_df.loc[amazon_df.genero=='Action, Animation',['genero']]='Acao, Animacao'
amazon_df.loc[amazon_df.genero=='Comedy, Drama, Science Fiction',['genero']]='Comedia, Drama, Ficcao Cientifica'
amazon_df.loc[amazon_df.genero=='Adventure, Kids',['genero']]='Aventura, Infantil'
amazon_df.loc[amazon_df.genero=='Horror, Suspense',['genero']]='Terror, Suspense'
amazon_df.loc[amazon_df.genero=='Action, Animation',['genero']]='Acao, Animacao'
amazon_df.loc[amazon_df.genero=='Science Fiction, Special Interest',['genero']]='Ficcao Cientifica, Conteudo Especial'
amazon_df.loc[amazon_df.genero=='Talk Show',['genero']]='Entrevista'
amazon_df.loc[amazon_df.genero=='Action, Animation',['genero']]='Acao, Animacao'
amazon_df.loc[amazon_df.genero=='Sports',['genero']]='Esporte'
amazon_df.loc[amazon_df.genero=='Kids',['genero']]='Infantil'
amazon_df.loc[amazon_df.genero=='Entertainment',['genero']]='Entretenimento'
amazon_df.loc[amazon_df.genero=='Animation, Anime, Fantasy',['genero']]='Animacao, Anime, Fantasia'
amazon_df.loc[amazon_df.genero=='Anime, Kids',['genero']]='Anime, Infantil'
amazon_df.loc[amazon_df.genero=='Animation',['genero']]='Animacao'
amazon_df.loc[amazon_df.genero=='Variety',['genero']]='Variedade'
amazon_df.loc[amazon_df.genero=='Action, Drama',['genero']]='Acao, Drama'
amazon_df.loc[amazon_df.genero=='Sports',['genero']]='Esporte'
amazon_df.loc[amazon_df.genero=='Kids',['genero']]='Infantil'
amazon_df.loc[amazon_df.genero=='Action, Adventure, Comedy',['genero']]='Acao, Aventura Comedia'
amazon_df.loc[amazon_df.genero=='Arts, Entertainment, and Culture, Documentary',['genero']]='Arte, Entretenimento, Cultura, Documentario'
amazon_df.loc[amazon_df.genero=='Documentary, LGBTQ',['genero']]='Documentario, LGBTQ'
amazon_df.loc[amazon_df.genero=='Culture',['genero']]='Cultura'
amazon_df.loc[amazon_df.genero=='Horror, Science Fiction',['genero']]='Terror, Ficcao Cientifica'
amazon_df.loc[amazon_df.genero=='Documentary, Sports',['genero']]='Documentario, Esporte'
amazon_df.loc[amazon_df.genero == "Documentary, Special Interest",['genero']] = 'Documentario, Conteudo Especial'
amazon_df.loc[amazon_df.genero == "Action, Science Fiction, Suspense",['genero']] = 'Acao, Ficcao Cientifica, Suspense'
amazon_df.loc[amazon_df.genero == "Kids, Special Interest",['genero']] = 'Infantil, Conteudo Especial'
amazon_df.loc[amazon_df.genero == "Comedy, Talk Show and Variety",['genero']] = 'Comedia, Entrevista e variedade'
amazon_df.loc[amazon_df.genero == "Science Fiction",['genero']] = 'Ficcao Cientifica'
amazon_df.loc[amazon_df.genero == 'Action, Anime, Comedy',['genero']] = 'Acao, Anime, Comedia'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Comedy, Drama',['genero']] = 'Artes, Entretenimento e Cultura, Comedia, Drama'
amazon_df.loc[amazon_df.genero == 'TV Shows',['genero']] = 'Series'
amazon_df.loc[amazon_df.genero == 'Action, Adventure, Animation',['genero']] = 'Acao, Aventura, Animacao'
amazon_df.loc[amazon_df.genero == 'Music Videos and Concerts',['genero']] = 'Musica, Videos e Concertos'
amazon_df.loc[amazon_df.genero == 'Fitness, Special Interest',['genero']] = 'Fitness, Conteudo Especial'
amazon_df.loc[amazon_df.genero == 'Faith and Spirituality, Special Interest',['genero']] = 'Fe e Espiritualidade, Conteudo Especial'
amazon_df.loc[amazon_df.genero == 'Special Interest',['genero']] = 'Conteudo Especial'
amazon_df.loc[amazon_df.genero == 'Action, Military and War, Western',['genero']] = 'Acao, Militar e Guerra, Ocidental'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Comedy, Talk Show and Variety',['genero']] = 'Artes, Entretenimento e Cultura, Comedia, Entrevistas e Variedade'
amazon_df.loc[amazon_df.genero == 'Action, Western',['genero']] = 'Acao, Ocidental'
amazon_df.loc[amazon_df.genero == 'Documentary, Science Fiction',['genero']] = 'Documentario e Ficcao Cientifica'
amazon_df.loc[amazon_df.genero == 'Adventure, Animation, Kids',['genero']] = 'Aventura, Animacao, Infantil'
amazon_df.loc[amazon_df.genero == 'Fantasy, Science Fiction, Suspense',['genero']] = 'Fantasia, Ficcao Cientifica, Suspense'
amazon_df.loc[amazon_df.genero == 'Action',['genero']] = 'Acao'
amazon_df.loc[amazon_df.genero == 'Documentary, LGBTQ',['genero']] = 'Documentario, LGBTQ'
amazon_df.loc[amazon_df.genero == 'Comedy, Special Interest, Unscripted',['genero']] = 'Comedia, Conteudo Especial, Improvisado'
amazon_df.loc[amazon_df.genero == 'Unscripted',['genero']] = 'Improvisado'
amazon_df.loc[amazon_df.genero == 'Documentary, Military and War',['genero']] = 'Documentario, Militar e Guerra'
amazon_df.loc[amazon_df.genero == 'Animation, Kids',['genero']] = 'Animacao, Infantil'
amazon_df.loc[amazon_df.genero == 'Action, Drama',['genero']] = 'Acao, Drama'
amazon_df.loc[amazon_df.genero == 'Comedy, Documentary, Unscripted', ['genero']] = 'Comedia, Documentario, Improvisado'
amazon_df.loc[amazon_df.genero == 'Western',['genero']] = 'Ocidental'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Comedy',['genero']] = 'Artes, Entretenimento e Cultura, Comedia'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Comedy, Special Interest',['genero']] = 'Artes, Entretenimento e Cultura, Comedia, Conteudo Especial'
amazon_df.loc[amazon_df.genero == 'Documentary, International, Special Interest',['genero']] = 'Documentario, Internacional, Conteudo Especial'
amazon_df.loc[amazon_df.genero == 'Comedy, Science Fiction',['genero']] = 'Comedia, Ficcao Cientifica'
amazon_df.loc[amazon_df.genero == 'Action, Drama, Sports',['genero']] = 'Acao, Drama, Esportes'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture',['genero']] = 'Artes, Entretenimento e Cultura'
amazon_df.loc[amazon_df.genero == 'Drama, Special Interest',['genero']] = 'Drama, Conteudo Especial'
amazon_df.loc[amazon_df.genero == 'Action, Science Fiction',['genero']] = 'Acao, Ficcao Cientifica'
amazon_df.loc[amazon_df.genero == 'Documentary, Faith and Spirituality, Special Interest',['genero']] = 'Documentario, Fe e Espiritualidade, Conteudo Especial'
amazon_df.loc[amazon_df.genero == 'Action, Drama, Special Interest',['genero']] = 'Acao, Drama, Conteudo Especial'
amazon_df.loc[amazon_df.genero == 'Drama, Young Adult Audience',['genero']] = 'Drama, Publico Jovem Adulto'
amazon_df.loc[amazon_df.genero == 'Drama, Military and War',['genero']] = 'Drama, Militar e Guerra'
amazon_df.loc[amazon_df.genero == 'Arthouse, Drama',['genero']] = 'Casa de Arte, Drama'
amazon_df.loc[amazon_df.genero == 'Documentary, Kids, Special Interest',['genero']] = 'Documentario, Infantil, Conteudo Especial'
amazon_df.loc[amazon_df.genero == 'Military and War, Western',['genero']] = 'Militar e Guerra, Ocidental'
amazon_df.loc[amazon_df.genero == 'Action, Comedy, Drama',['genero']] = 'Acao, Comedia, Drama'
amazon_df.loc[amazon_df.genero == 'Comedy, International',['genero']] = 'Comedia, Internacional'
amazon_df.loc[amazon_df.genero == 'Drama, Horror, Suspense',['genero']] = 'Drama, Terror, Suspense'
amazon_df.loc[amazon_df.genero == 'Animation, Comedy, Kids',['genero']] = 'Animacao, Comedia, Infantil'
amazon_df.loc[amazon_df.genero == 'Action, Suspense',['genero']] = 'Acao, Suspense'
amazon_df.loc[amazon_df.genero == 'Documentary, Drama, Unscripted',['genero']] = 'Documnetario, Drama, Improvisado '
amazon_df.loc[amazon_df.genero == 'Action, Drama, International',['genero']] = 'Acao, Drama, Internacional'
amazon_df.loc[amazon_df.genero == 'Drama, Science Fiction, Suspense',['genero']] = 'Drama, Ficcao Cientifica, Suspense'
amazon_df.loc[amazon_df.genero == 'International, Suspense',['genero']] = 'Internacional, Suspense'
amazon_df.loc[amazon_df.genero == 'Action, International',['genero']] = 'Acao, Internacional'
amazon_df.loc[amazon_df.genero == 'Anime, Drama, International',['genero']] = 'Anime, Drama, Internacional'
amazon_df.loc[amazon_df.genero == 'Action, Documentary, Sports',['genero']] = 'Acao, Documentario, Esportes'
amazon_df.loc[amazon_df.genero == 'Action, Adventure, Anime',['genero']] = 'Acao, Aventura, Anime'
amazon_df.loc[amazon_df.genero == 'Animation, Drama',['genero']] = 'Animacao, Drama'
amazon_df.loc[amazon_df.genero == 'Special Interest, Sports',['genero']] = 'Conteudo Especial, Esportes'
amazon_df.loc[amazon_df.genero == 'Horror',['genero']] = 'Terror'
amazon_df.loc[amazon_df.genero == 'Action, Comedy',['genero']] = 'Acao, Comedia'
amazon_df.loc[amazon_df.genero == 'Arthouse, Comedy, Drama',['genero']] = 'Casa de Arte, Comedia, Drama'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Documentary, Special Interest',['genero']] = 'Artes, Entretenimento e Cultura, Documentario, Interesse Especial'
amazon_df.loc[amazon_df.genero == 'Animation, International, Kids',['genero']] = 'Animacao, Internacional, Infantil'
amazon_df.loc[amazon_df.genero == 'Comedy, Drama, LGBTQ',['genero']] = 'Comedia, Drama, LGBTQ'
amazon_df.loc[amazon_df.genero == 'Action, Adventure',['genero']] = 'Acao, Aventura'
amazon_df.loc[amazon_df.genero == 'Comedy, Drama, Sports',['genero']] = 'Comedia, Drama, Esportes'
amazon_df.loc[amazon_df.genero == 'Comedy, Romance',['genero']] = 'Comedia, Romance'
amazon_df.loc[amazon_df.genero == 'Documentary, Drama, Sports',['genero']] = 'Documentario, Drama, Esportes'
amazon_df.loc[amazon_df.genero == 'Drama, International, Romance',['genero']] = 'Drama, Internacional, Romance'
amazon_df.loc[amazon_df.genero == 'Comedy, Drama, Romance',['genero']] = 'Comedia, Drama, Romance'
amazon_df.loc[amazon_df.genero == 'Animation, Anime, Drama',['genero']] = 'Animacao, Anime, Drama'
amazon_df.loc[amazon_df.genero == 'Action, Drama, Kids',['genero']] = 'Acao, Drama, Infantil'
amazon_df.loc[amazon_df.genero == 'Fantasy, Horror, International',['genero']] = 'Fantasia, Terror, Internacional'
amazon_df.loc[amazon_df.genero == 'Action, Animation, Kids',['genero']] = 'Acao, Animacao, Infantil'
amazon_df.loc[amazon_df.genero == 'Drama, Faith and Spirituality, Special Interest',['genero']] = 'Drama, Fe e Espiritualidade, Conteudo Especial'
amazon_df.loc[amazon_df.genero == 'Action, Drama, Western',['genero']] = 'Acao, Drama, Ocidental'
amazon_df.loc[amazon_df.genero == 'Action, Adventure, Suspense',['genero']] = 'Acao, Aventura, Suspense'
amazon_df.loc[amazon_df.genero == 'Documentary, International',['genero']] = 'Documentario, Internacional'
amazon_df.loc[amazon_df.genero == 'Adventure, Fantasy',['genero']] = 'Aventura, Fantasia'
amazon_df.loc[amazon_df.genero == 'Action, Romance, Western',['genero']] = 'Acao, Romance, Ocidental'
amazon_df.loc[amazon_df.genero == 'Documentary, Talk Show and Variety',['genero']] = 'Documentario, Entrevistas e variedade'
amazon_df.loc[amazon_df.genero == 'Documentary, Drama, LGBTQ',['genero']] = 'Documentario, Drama, LGBTQ'
amazon_df.loc[amazon_df.genero == 'Documentary, Sports, Unscripted',['genero']] = 'Documentario, Esportes, Improvisado'
amazon_df.loc[amazon_df.genero == 'Comedy, Kids',['genero']] = 'Comedia, Infantil'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Drama',['genero']] = 'Artes, Entretenimento e Cultura, Drama'
amazon_df.loc[amazon_df.genero == 'Music Videos and Concerts, Special Interest',['genero']] = 'Musica, Videos e Concertos, Conteudo Especial'
amazon_df.loc[amazon_df.genero == 'Drama, Faith and Spirituality, Romance',['genero']] = 'Drama, Fe e Espiritualidade, Romance'
amazon_df.loc[amazon_df.genero == 'Drama, Faith and Spirituality, LGBTQ',['genero']] = 'Drama, Fe e Espiritualidade, LGBTQ'
amazon_df.loc[amazon_df.genero == 'Drama, Suspense, Young Adult Audience',['genero']] = 'Drama, Suspense, Publico Jovem Adulto'
amazon_df.loc[amazon_df.genero == 'Drama, LGBTQ, Unscripted',['genero']] = 'Drama, LGBTQ, Improvisado'
amazon_df.loc[amazon_df.genero == 'Documentary, Music Videos and Concerts',['genero']] = 'Documentario, Musica, Videos e Concertos'
amazon_df.loc[amazon_df.genero == 'Drama, Faith and Spirituality',['genero']] = 'Drama, Fe e Espiritualidade'
amazon_df.loc[amazon_df.genero == 'Special Interest, Unscripted',['genero']] = 'Conteudo Especial, Improvisado'
amazon_df.loc[amazon_df.genero == 'Action, Fantasy',['genero']] = 'Acao, Fantasia'
amazon_df.loc[amazon_df.genero == 'Action, Drama, Horror',['genero']] = 'Acao, Drama, Terror'
amazon_df.loc[amazon_df.genero == 'International, Music Videos and Concerts, Unscripted',['genero']] = 'Internacional, Musica, Videos e Concertos, Improvisado'
amazon_df.loc[amazon_df.genero == 'Action, Kids',['genero']] = 'Acao, Infantil'
amazon_df.loc[amazon_df.genero == 'Action, Drama, Romance',['genero']] = 'Acao, Drama, Romance'
amazon_df.loc[amazon_df.genero == 'Drama, Sports',['genero']] = 'Drama, Esportes'
amazon_df.loc[amazon_df.genero == 'Documentary, Drama, Special Interest',['genero']] = 'Documentario, Drama, Conteudo Especial'
amazon_df.loc[amazon_df.genero == 'Animation, Comedy, Science Fiction',['genero']] = 'Animacao, Comedia, Ficcao Cientifica'
amazon_df.loc[amazon_df.genero == 'Aventura, Drama',['genero']] = 'Aventura, Drama'
amazon_df.loc[amazon_df.genero == 'Comedy, Western',['genero']] = 'Comedia, Ocidental'
amazon_df.loc[amazon_df.genero == 'Animation, Kids, Special Interest',['genero']] = 'Animacao, Infantil, Conteudo Especial'
amazon_df.loc[amazon_df.genero == 'Drama, Unscripted, Young Adult Audience',['genero']] = 'Drama, Improvisado, Publico Jovem Adulto'
amazon_df.loc[amazon_df.genero == 'Drama, Kids',['genero']] = 'Drama, Infantil'
amazon_df.loc[amazon_df.genero == 'Documentary, Horror',['genero']] = 'Documentario, Terror'
amazon_df.loc[amazon_df.genero == 'Action, Comedy, Horror',['genero']] = 'Acao, Comedia, Terror'
amazon_df.loc[amazon_df.genero == 'Comedy, Suspense',['genero']] = 'Comedia, Suspense'
amazon_df.loc[amazon_df.genero == 'Animation, Drama, Faith and Spirituality',['genero']] = 'Animacao, Drama, Fe e Espiritualidade'
amazon_df.loc[amazon_df.genero == 'Drama, Historical',['genero']] = 'Drama, Historico'
amazon_df.loc[amazon_df.genero == 'Drama, Science Fiction',['genero']] = 'Drama, Ficcao Cientifica'
amazon_df.loc[amazon_df.genero == 'Comedy, Documentary, Special Interest',['genero']] = 'Comedia, Documentario, Conteudo Especial'
amazon_df.loc[amazon_df.genero == 'Action, Drama, Historical',['genero']] = 'Acao, Drama, Historico'
amazon_df.loc[amazon_df.genero == 'Animation, Anime, Kids',['genero']] = 'Animacao, Anime, Infantil'
amazon_df.loc[amazon_df.genero == 'Romance, Science Fiction',['genero']] = 'Romance, Ficcao Cientifica'
amazon_df.loc[amazon_df.genero == 'Drama, Historical, Military and War',['genero']] = 'Drama, Historico, Militar e Guerra'
amazon_df.loc[amazon_df.genero == 'Drama, Faith and Spirituality, Suspense',['genero']] = 'Drama, Fe e Espiritualidade, Suspense'
amazon_df.loc[amazon_df.genero == 'Arthouse, Arts, Entertainment, and Culture, Drama',['genero']] = 'Casa de Arte, Artes, Entretenimento e Cultura, Drama'
amazon_df.loc[amazon_df.genero == 'Documentary, Special Interest, Unscripted',['genero']] = 'Drama, Fe e Espiritualidade, Suspense'
amazon_df.loc[amazon_df.genero == 'Adventure',['genero']] = 'Aventura'
amazon_df.loc[amazon_df.genero == 'Comedy, Young Adult Audience',['genero']] = 'Comedia, Publico Jovem Adulto'
amazon_df.loc[amazon_df.genero == 'Action, Special Interest',['genero']] = 'Acao, Conteudo Especial'
amazon_df.loc[amazon_df.genero == 'Action, Documentary, Unscripted',['genero']] = 'Acao, Documentario, Improvisado'
amazon_df.loc[amazon_df.genero == 'Documentary, Special Interest, Sports',['genero']] = 'Documentario, Conteudo Especial, Esportes'
amazon_df.loc[amazon_df.genero == 'Kids, Special Interest, Unscripted',['genero']] = 'Infantil, Conteudo Especial, Improvisado'
amazon_df.loc[amazon_df.genero == 'Comedy, Documentary, Sports',['genero']] = 'Comedia, Documentario, Esportes'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Documentary, Music Videos and Concerts',['genero']] = 'Artes, Entretenimento e Cultura, Documentario, Musica, Videos e Concertos'
amazon_df.loc[amazon_df.genero == 'Horror, Unscripted',['genero']] = 'Terror, Improvisado'
amazon_df.loc[amazon_df.genero == 'Documentary, Drama',['genero']] = 'Documentario, Drama'
amazon_df.loc[amazon_df.genero == 'Suspense, Western',['genero']] = 'Suspense, Ocidental'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Comedy, Romance',['genero']] = 'Artes, Entretenimento e Cultura, Comedia, Romance'
amazon_df.loc[amazon_df.genero == 'Arthouse, Arts, Entertainment, and Culture, Comedy',['genero']] = 'Casa de Arte, Artes, Entretenimento e Cutura, Comedia'
amazon_df.loc[amazon_df.genero == 'Faith and Spirituality',['genero']] = 'Fe e Espiritualidade'
amazon_df.loc[amazon_df.genero == 'Drama, Historical, International',['genero']] = 'Drama, Historico, Internacional'
amazon_df.loc[amazon_df.genero == 'Action, Kids, Science Fiction',['genero']] = 'Acao, Infantil, Ficcao Cientifica'
amazon_df.loc[amazon_df.genero == 'Action, Military and War',['genero']] = 'Acao, Militar e Guerra'
amazon_df.loc[amazon_df.genero == 'Action, Horror',['genero']] = 'Acao, Terror'
amazon_df.loc[amazon_df.genero == 'Young Adult Audience',['genero']] = 'Publico Jovem Adulto'
amazon_df.loc[amazon_df.genero == 'Horror, Young Adult Audience',['genero']] = 'Terror, Publico Jovem Adulto'
amazon_df.loc[amazon_df.genero == 'Action, Adventure, Drama',['genero']] = 'Acao, Aventura, Drama'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Drama, Romance',['genero']] = 'Artes, Entretenimento e Cultura, Drama, Romance'
amazon_df.loc[amazon_df.genero == 'Action, Horror, Suspense',['genero']] = 'Acao, Terror, Suspense'
amazon_df.loc[amazon_df.genero == 'Comedy, Drama, Kids',['genero']] = 'Comedia, Drama, Infantil'
amazon_df.loc[amazon_df.genero == 'Action, Fantasy, Science Fiction',['genero']] = 'Acao, Fantasia, Ficcao Cientifica'
amazon_df.loc[amazon_df.genero == 'Documentary, Faith and Spirituality',['genero']] = 'Documentario, Fe e Espiritualidade'
amazon_df.loc[amazon_df.genero == 'Arthouse, Arts, Entertainment, and Culture, Documentary',['genero']] = 'Casa de Arte, Entretenimento e Cultura, Documentario'
amazon_df.loc[amazon_df.genero == 'Drama, Special Interest, Unscripted',['genero']] = 'Drama, Conteudo Especial, Improvisado'
amazon_df.loc[amazon_df.genero == 'Adventure, Drama, Military and War',['genero']] = 'Aventura, Drama, Militar e Guerra'
amazon_df.loc[amazon_df.genero == 'Documentary, Unscripted',['genero']] = 'Documentario, Improvisado'
amazon_df.loc[amazon_df.genero == 'Adventure, Documentary, Military and War',['genero']] = 'Aventura, Documentario, Militar e Guerra'
amazon_df.loc[amazon_df.genero == 'Documentary, Kids',['genero']] = 'Documentario, Infantil'
amazon_df.loc[amazon_df.genero == 'Military and War, Special Interest',['genero']] = 'Militar e Guerra, Conteudo Especial'
amazon_df.loc[amazon_df.genero == 'Arthouse, Drama, Suspense',['genero']] = 'Casa de Arte, Drama, Suspense'
amazon_df.loc[amazon_df.genero == 'Comedy, Drama, International',['genero']] = 'Comedia, Drama, Internacional'
amazon_df.loc[amazon_df.genero == 'Comedy, International, Young Adult Audience',['genero']] = 'Comedia, Internacional, Publico Jovem Adulto'
amazon_df.loc[amazon_df.genero == 'Action, Anime, Fantasy',['genero']] = 'Acao, Anime, Fantasia'
amazon_df.loc[amazon_df.genero == 'Documentary, Special Interest, Suspense',['genero']] = 'Documentario, Conteudo Epecial, Suspense'
amazon_df.loc[amazon_df.genero == 'Action, Adventure, Documentary',['genero']] = 'Acao, Aventura, Documentario'
amazon_df.loc[amazon_df.genero == 'Animation, Faith and Spirituality, Kids',['genero']] = 'Animacao, Fe e Espiritualidade, Infantil'
amazon_df.loc[amazon_df.genero == 'Animation, Documentary, Kids',['genero']] = 'Animacao, Documentario, Infantil'
amazon_df.loc[amazon_df.genero == 'Animation, Kids, Sports',['genero']] = 'Animacao, Infantil, Esportes'
amazon_df.loc[amazon_df.genero == 'Arthouse, Drama, Romance',['genero']] = 'Casa de Arte, Drama, Romance'
amazon_df.loc[amazon_df.genero == 'Action, Young Adult Audience',['genero']] = 'Acao, Publico Jovem Adulto'
amazon_df.loc[amazon_df.genero == 'Drama, Faith and Spirituality, Kids',['genero']] = 'Drama, Fe e Espiritualidade, Infantil'
amazon_df.loc[amazon_df.genero == 'Drama, Science Fiction, Special Interest',['genero']] = 'Drama, Ficcao Cientifica, Conteudo Especial'
amazon_df.loc[amazon_df.genero == 'Action, Adventure, Science Fiction',['genero']] = 'Acao, Aventura, Ficcao Cientifica'
amazon_df.loc[amazon_df.genero == 'Adventure, Drama, Western',['genero']] = 'Aventura, Drama, Ocidental'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Comedy, Military and War',['genero']] = 'Artes, Entretenimento e Cultura, Comedia, Militar e Guerra'
amazon_df.loc[amazon_df.genero == 'Drama, Horror',['genero']] = 'Drama, Terror'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Drama, Western',['genero']] = 'Artes, Enttetenimento e Cultura, Drama, Ocidental'
amazon_df.loc[amazon_df.genero == 'Comedy, Drama, Suspense',['genero']] = 'Comedia, Drama, Suspense'

amazon_df.loc[amazon_df.genero == 'Animation, Comedy',['genero']] = 'Animacao, Comedia'
amazon_df.loc[amazon_df.genero == 'Kids, Science Fiction',['genero']] = 'Infantil, Ficcao Cientifica'
amazon_df.loc[amazon_df.genero == 'Adventure, Comedy, Kids',['genero']] = 'Aventura, Comedia, Infantil'
amazon_df.loc[amazon_df.genero == 'Comedy, Drama, Young Adult Audience',['genero']] = 'Comedia, Drama, Publico Jovem Adulto'
amazon_df.loc[amazon_df.genero == 'Fantasy, Science Fiction',['genero']] = 'Fantasia, Ficcao Cientifica'
amazon_df.loc[amazon_df.genero == 'International, Unscripted',['genero']] = 'Internacional, Improvisado'
amazon_df.loc[amazon_df.genero == 'Arthouse, Comedy',['genero']] = 'Casa de Arte, Comedia'
amazon_df.loc[amazon_df.genero == 'Sports, Unscripted',['genero']] = 'Esportes, Improvisado'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Comedy, Documentary',['genero']] = 'Artes, Entretenimento e Cultura, Comedia, Documentario'
amazon_df.loc[amazon_df.genero == 'Adventure, Drama, Suspense',['genero']] = 'Aventura, Drama, Suspense'
amazon_df.loc[amazon_df.genero == 'Action, Horror, Science Fiction',['genero']] = 'Acao, Terror, Ficcao Cientifica'
amazon_df.loc[amazon_df.genero == 'Comedy, International, Unscripted',['genero']] = 'Comedia, Internacional, Improvisado'
amazon_df.loc[amazon_df.genero == 'Drama, Kids, Romance',['genero']] = 'Drama, Infantil, Romance'
amazon_df.loc[amazon_df.genero == 'International, Special Interest',['genero']] = 'Internacional, Conteudo Especial'

amazon_df.loc[amazon_df.genero == 'Action, Adventure, International',['genero']] = 'Acao, Aventura, Internacional'
amazon_df.loc[amazon_df.genero == 'Documentary, Military and War, Special Interest',['genero']] = 'Documentario, Militar e Guerra, Conteudo Especial'
amazon_df.loc[amazon_df.genero == 'Adventure, Documentary, Special Interest',['genero']] = 'Adentura, Documentario, Conteudo Especial'
amazon_df.loc[amazon_df.genero == 'Action, Military and War, Suspense',['genero']] = 'Acao, Militar e Guerra, Suspense'
amazon_df.loc[amazon_df.genero == 'Horror, Science Fiction, Suspense',['genero']] = 'Terror, Ficcao Cientifica, Suspense'
amazon_df.loc[amazon_df.genero == 'Adventure, Drama, Faith and Spirituality',['genero']] = 'Aventura, Drama, Fe e Espiritualidade'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Drama, Kids',['genero']] = 'Artes, Entretenimento e Cultura, Drama, Infantil'
amazon_df.loc[amazon_df.genero == 'Action, Documentary, Special Interest',['genero']] = 'Acao, Documentario, Conteudo Especial'
amazon_df.loc[amazon_df.genero == 'Animation, Anime, Horror',['genero']] = 'Animacao, Anime, Terror'
amazon_df.loc[amazon_df.genero == 'Arthouse, Special Interest',['genero']] = 'Casa de Arte, Conteúdo Especial'
amazon_df.loc[amazon_df.genero == 'Action, International, Suspense',['genero']] = 'Acao, Internacional, Suspense'
amazon_df.loc[amazon_df.genero == 'Action, Sports',['genero']] = 'Acao, Esportes'
amazon_df.loc[amazon_df.genero == 'Animation, Anime',['genero']] = 'Animacao, Anime'

amazon_df.loc[amazon_df.genero == 'Drama, Historical, Suspense',['genero']] = 'Drama, Historico, Suspense'
amazon_df.loc[amazon_df.genero == 'Talk Show and Variety',['genero']] = 'Entrevistas e Variedade'
amazon_df.loc[amazon_df.genero == 'Drama, Unscripted',['genero']] = 'Drama, Improvisado'
amazon_df.loc[amazon_df.genero == 'Drama, Horror, Special Interest',['genero']] = 'Drama, Terror, Conteúdo Especial'
amazon_df.loc[amazon_df.genero == 'LGBTQ, Special Interest',['genero']] = 'LGBTQ, Conteudo Especial'
amazon_df.loc[amazon_df.genero == 'Action, Romance',['genero']] = 'Acao, Romance'
amazon_df.loc[amazon_df.genero == 'Horror, International',['genero']] = 'Terror, Internacional'
amazon_df.loc[amazon_df.genero == 'Drama, Fantasy, Suspense',['genero']] = 'Drama, Fantasia, Suspense'
amazon_df.loc[amazon_df.genero == 'Comedy, Kids, Talk Show and Variety',['genero']] = 'Comedia, Infantil, Entrevistas e Variedade'
amazon_df.loc[amazon_df.genero == 'Drama, Fantasy, Science Fiction',['genero']] = 'Drama, Fantasia, Ficca Cientifica'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Comedy, International',['genero']] = 'Artes, Entretenimento e Cultura, Comedia, Internacional'
amazon_df.loc[amazon_df.genero == 'Action, Documentary',['genero']] = 'Internacional, Conteudo Especial'
amazon_df.loc[amazon_df.genero == 'Action, Drama, Science Fiction',['genero']] = 'Acao, Drama, Ficcao Cientifica'
amazon_df.loc[amazon_df.genero == 'Kids, Music Videos and Concerts',['genero']] = 'Infantil, Musica Videos e Concertos'
amazon_df.loc[amazon_df.genero == 'Documentary, Horror, Special Interest',['genero']] = 'Documentario, Terror, Conteudo Especial'


amazon_df.loc[amazon_df.genero == 'Comedy, LGBTQ, Romance',['genero']] = 'Comedia, LGBTQ, Romance'
amazon_df.loc[amazon_df.genero == 'Action, Comedy, Western',['genero']] = 'Acao, Comedia, Ocidental'
amazon_df.loc[amazon_df.genero == 'Comedy, International, Romance',['genero']] = 'Comedia, Internacional, Romance'
amazon_df.loc[amazon_df.genero == 'Documentary, Suspense',['genero']] = 'Documentario, Suspense'
amazon_df.loc[amazon_df.genero == 'Action, Arthouse, Drama',['genero']] = 'Acao, Casa de Arte, Drama'
amazon_df.loc[amazon_df.genero == 'International',['genero']] = 'Internacional'
amazon_df.loc[amazon_df.genero == 'Action, Documentary, Faith and Spirituality',['genero']] = 'Acao, Documentario, Fe e Espiritualidade'
amazon_df.loc[amazon_df.genero == 'Drama, Kids, LGBTQ',['genero']] = 'Drama, Infantil, LGBTQ'
amazon_df.loc[amazon_df.genero == 'Drama, Fantasy, Historical',['genero']] = 'Drama, Fantasia, Historico'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Music Videos and Concerts',['genero']] = 'Artes, Entretenimento e Cultura, Musica Videos e Concertos'
amazon_df.loc[amazon_df.genero == 'Action, Suspense, Young Adult Audience',['genero']] = 'Acao, Suspense, Publico Jovem Adulto'
amazon_df.loc[amazon_df.genero == 'Action, Anime, International',['genero']] = 'Acao, Anime, Internacional'
amazon_df.loc[amazon_df.genero == 'International, Science Fiction, Suspense',['genero']] = 'Internacional, Ficcao Cientifica, Suspense'




In [32]:
amazon_df.loc[amazon_df.genero == 'Action, Arthouse, Arts, Entertainment, and Culture',['genero']] = 'Acao, Casa de Arte, Artes, Entretenimento e Cultura'
amazon_df.loc[amazon_df.genero == 'Comedy, International, Special Interest',['genero']] = 'Comedia, Internacional, Conteudo Especial'
amazon_df.loc[amazon_df.genero == 'Comedy, LGBTQ',['genero']] = 'Comedia, LGBTQ'
amazon_df.loc[amazon_df.genero == 'Adventure, Comedy, Drama',['genero']] = 'Aventura, Comedia, Drama'
amazon_df.loc[amazon_df.genero == 'Fantasy, Kids, Suspense',['genero']] = 'Fantasia, Infantil, Suspense'
amazon_df.loc[amazon_df.genero == 'Arthouse, Comedy, Romance',['genero']] = 'Casa de Arte, Comedia, Romance'
amazon_df.loc[amazon_df.genero == 'Comedy, Sports',['genero']] = 'Comedia, Esportes'
amazon_df.loc[amazon_df.genero == 'Action, Comedy, Fantasy',['genero']] = 'Acao, Comedia, Fantasia'

amazon_df.loc[amazon_df.genero == 'Action, Sports, Suspense',['genero']] = 'Acao, Esportes, Suspense'
amazon_df.loc[amazon_df.genero == 'Action, Animation, Anime',['genero']] = 'Acao, Animacao, Anime'
amazon_df.loc[amazon_df.genero == 'Anime, Comedy, International',['genero']] = 'Anime, Comedia, Internacional'
amazon_df.loc[amazon_df.genero == 'Action, Adventure, Kids',['genero']] = 'Action, Aventura, Infantil'
amazon_df.loc[amazon_df.genero == 'Action, Comedy, Documentary',['genero']] = 'Acao, Comedia, Documentario'


amazon_df.loc[amazon_df.genero == 'Drama, Fantasy, Kids',['genero']] = 'Drama, Fantasia, Infantil'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Comedy, Unscripted',['genero']] = 'Artes, Entretenimento e Cultura, Comedia, Improvisso'
amazon_df.loc[amazon_df.genero == 'Drama, Horror, Science Fiction',['genero']] = 'Drama, Internacional, Esportes'
amazon_df.loc[amazon_df.genero == 'Drama, International, Sports',['genero']] = 'Drama, Internacional, Esportes'
amazon_df.loc[amazon_df.genero == 'International, Sports',['genero']] = 'Internacional, Esportes'
amazon_df.loc[amazon_df.genero == 'Science Fiction, Suspense',['genero']] = 'Ficcao Cientifica, Suspense'
amazon_df.loc[amazon_df.genero == 'Action, Special Interest, Sports',['genero']] = 'Acao, Conteudo Especial, Esportes'
amazon_df.loc[amazon_df.genero == 'LGBTQ, Special Interest, Unscripted',['genero']] = 'LGBTQ, Conteudo Especial, Improvisado'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Comedy, LGBTQ',['genero']] = 'Artes, Entretenimento e Cultura, Comedia, LGBTQ'
amazon_df.loc[amazon_df.genero == 'Arthouse, Drama, International',['genero']] = 'Casa de Arte, Drama, Internacional'
amazon_df.loc[amazon_df.genero == 'Action, Suspense, Unscripted',['genero']] = 'Acao, Suspense, Improvisado'
amazon_df.loc[amazon_df.genero == 'Action, Comedy, LGBTQ',['genero']] = 'Acao, Comedia, LGBTQ'
amazon_df.loc[amazon_df.genero == 'Special Interest, Western',['genero']] = 'Conteudo Especial, Ocidental'
amazon_df.loc[amazon_df.genero == 'Kids, Western',['genero']] = 'Infantil, Ocidental'
amazon_df.loc[amazon_df.genero == 'Comedy, Drama, Fantasy',['genero']] = 'Comedia, Drama, Fantasia'
amazon_df.loc[amazon_df.genero == 'Comedy, Special Interest',['genero']] = 'Comedia, Conteudo Especial'
amazon_df.loc[amazon_df.genero == 'Action, Drama, Military and War',['genero']] = 'Acao, Drama, Militar e Guerra'
amazon_df.loc[amazon_df.genero == 'International, Romance',['genero']] = 'Internacional, Romance'
amazon_df.loc[amazon_df.genero == 'Adventure, Drama, Kids',['genero']] = 'Aventura, Drama, Infantil'
amazon_df.loc[amazon_df.genero == 'Documentary, Drama, Horror',['genero']] = 'Documentario, Drama, Terror'
amazon_df.loc[amazon_df.genero == 'Science Fiction, Suspense, Unscripted',['genero']] = 'Ficcao Cientifica, Suspense, Improvisado'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Drama, Young Adult Audience',['genero']] = 'Artes, Entretenimento e Cultura, Drama, Publico Jovem Adulto'



In [33]:
amazon_df.loc[amazon_df.genero == 'Fitness, Kids, Special Interest',['genero']] = 'Fitness, Infantil, Conteudo Especial'
amazon_df.loc[amazon_df.genero == 'Documentary, Music Videos and Concerts, Special Interest',['genero']] = 'Documentario, Musica Videos e Concertos, Conteudo Especial'
amazon_df.loc[amazon_df.genero == 'Comedy, Horror, Suspense',['genero']] = 'Comedia, Terror, Suspense'
amazon_df.loc[amazon_df.genero == 'Horror, Suspense, Young Adult Audience',['genero']] = 'Terror, Suspense, Publico Jovem Adulto'
amazon_df.loc[amazon_df.genero == 'Adventure, Comedy',['genero']] = 'Aventura, Comedia'
amazon_df.loc[amazon_df.genero == 'Arthouse, Documentary',['genero']] = 'Casa de Arte, Documentario'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Drama, International',['genero']] = 'Artes, Entretenimento e Cultura, Drama, Internacional'
amazon_df.loc[amazon_df.genero == 'Action, Adventure, Western',['genero']] = 'Acao, Aventura, Ocidental'
amazon_df.loc[amazon_df.genero == 'Documentary, LGBTQ, Music Videos and Concerts',['genero']] = 'Documentario, LGBTQ, Musica Videos e Concertos'
amazon_df.loc[amazon_df.genero == 'Action, Unscripted',['genero']] = 'Acao, Improviso'
amazon_df.loc[amazon_df.genero == 'Drama, Horror, Western',['genero']] = 'Drama, Terror, Ocidental'
amazon_df.loc[amazon_df.genero == 'Documentary, Science Fiction, Special Interest',['genero']] = 'Documentario, Ficcao Cientifica, Conteudo Especial'
amazon_df.loc[amazon_df.genero == 'Adventure, Kids, Science Fiction',['genero']] = 'Aventura, Infantil, Ficcao Cientifica'
amazon_df.loc[amazon_df.genero == 'Drama, LGBTQ, Young Adult Audience',['genero']] = 'Drama, LGBTQ, Publico Jovem Adulto'
amazon_df.loc[amazon_df.genero == 'Action, Arts, Entertainment, and Culture, Comedy',['genero']] = 'Acao, Artes, Entretenimento e Cultura, Comedia'
amazon_df.loc[amazon_df.genero == 'Action, Adventure, Fantasy',['genero']] = 'Acao, Aventura, Fantasia'
amazon_df.loc[amazon_df.genero == 'Adventure, Sports',['genero']] = 'Aventura, Esportes'
amazon_df.loc[amazon_df.genero == 'Comedy, Fantasy, Kids',['genero']] = 'Comedia, Fantasia, Infantil'
amazon_df.loc[amazon_df.genero == 'Drama, Horror, Young Adult Audience',['genero']] = 'Drama, Terror, Publico Jovem Adulto'
amazon_df.loc[amazon_df.genero == 'Documentary, Drama, Suspense',['genero']] = 'Documentario, Drama, Suspense'
amazon_df.loc[amazon_df.genero == 'Historical, Horror, Western',['genero']] = 'Historico, Terror, Ocidental'
amazon_df.loc[amazon_df.genero == 'Action, Arts, Entertainment, and Culture, Drama',['genero']] = 'Acao, Artes, Entretenimento e Cultura, Drama'
amazon_df.loc[amazon_df.genero == 'Comedy, Historical',['genero']] = 'Comedia, Historio'
amazon_df.loc[amazon_df.genero == 'Fitness, Kids',['genero']] = 'Fitness, Infantil'
amazon_df.loc[amazon_df.genero == 'Action, Animation, Fantasy',['genero']] = 'Acao, Animacao, Fantasia'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, International, Unscripted',['genero']] = 'Artes, Entretenimento e Cultura, Internacional, Improviso'
amazon_df.loc[amazon_df.genero == 'Kids, Sports',['genero']] = 'Infantil, Esportes'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Comedy, Young Adult Audience',['genero']] = 'Artes, Entretenimento e Cultura, Comedia, Publico Jovem Adulto'


amazon_df.loc[amazon_df.genero == 'Documentary, Drama, Kids',['genero']] = 'Documentario, Drama, Infantil'
amazon_df.loc[amazon_df.genero == 'Anime, International, Kids',['genero']] = 'Anime, Internacional, Infantil'
amazon_df.loc[amazon_df.genero == 'Action, Comedy, Special Interest',['genero']] = 'Acao, Comedia, Conteudo Especial'
amazon_df.loc[amazon_df.genero == 'Action, Comedy, Military and War',['genero']] = 'Acao, Comedia, Militar e Guerra'
amazon_df.loc[amazon_df.genero == 'Fantasy, Horror, Science Fiction',['genero']] = 'Fantasia, Terror, Ficcao Cientifica'
amazon_df.loc[amazon_df.genero == 'Comedy, Music Videos and Concerts',['genero']] = 'Comedia, Musica Videos e Concertos'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Romance',['genero']] = 'Artes, Entretenimento e Cultura, Romance'
amazon_df.loc[amazon_df.genero == 'LGBTQ, Romance, Sports',['genero']] = 'LGBTQ, Romance, Esportes'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Documentary, LGBTQ',['genero']] = 'Artes, Entretenimento e Cultura, Documentario, LGBTQ'
amazon_df.loc[amazon_df.genero == 'Action, Comedy, International',['genero']] = 'Acao, Comedia, Internacional'
amazon_df.loc[amazon_df.genero == 'Adventure, International, Suspense',['genero']] = 'Aventura, Internacional, Suspense'
amazon_df.loc[amazon_df.genero == 'Action, Fantasy, Suspense',['genero']] = 'Acao, Fantasia, Suspense'
amazon_df.loc[amazon_df.genero == 'Animation, Arts, Entertainment, and Culture, Comedy',['genero']] = 'Animacao, Artes, Entretenimento e Cultura, Comedia'
amazon_df.loc[amazon_df.genero == 'Action, Kids, Suspense',['genero']] = 'Acao, Infantil, Suspense'
amazon_df.loc[amazon_df.genero == 'Action, Comedy, Kids',['genero']] = 'Acao, Comedia, Infantil'
amazon_df.loc[amazon_df.genero == 'Kids, LGBTQ, Romance',['genero']] = 'Infantil, LGBTQ, Romance'
amazon_df.loc[amazon_df.genero == 'Action, International, Science Fiction',['genero']] = 'Acao, Internacional, Ficcao Cientifica'
amazon_df.loc[amazon_df.genero == 'Adventure, Comedy, Fantasy',['genero']] = 'Aventura, Comedia, Fantasia'
amazon_df.loc[amazon_df.genero == 'Animation, Kids, Music Videos and Concerts',['genero']] = 'Animacao, Infantil, Musica Videos e Concertos'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Comedy, Science Fiction',['genero']] = 'Artes, Entretenimento e Cultura, Comedia, Ficcao Cientifica'
amazon_df.loc[amazon_df.genero == 'Adventure, Fantasy, Kids',['genero']] = 'Aventura, Fantasia, Infantil'


In [34]:
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, International',['genero']] = 'Artes, Entretenimento e Cultura, Internacional'
amazon_df.loc[amazon_df.genero == 'Action, Kids, Military and War',['genero']] = 'Acao, Infantil, Militar e Guerra'
amazon_df.loc[amazon_df.genero == 'Comedy, Horror, Young Adult Audience',['genero']] = 'Comedia, Terror, Publico Jovem Adulto'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Music Videos and Concerts, Special Interest',['genero']] = 'Artes, Entretenimento e Cultura, Musica Videos e Concertos, Conteudo Especial'
amazon_df.loc[amazon_df.genero == 'Historicals',['genero']] = 'Historico'
amazon_df.loc[amazon_df.genero == 'Action, Comedy, Science Fiction',['genero']] = 'Acao, Comedia, Ficca Cientifica'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Horror, Suspense',['genero']] = 'Artes, Entretenimento e Cultura, Terror, Suspense'
amazon_df.loc[amazon_df.genero == 'Comedy, Sports, Young Adult Audience',['genero']] = 'Comedia, Esportes, Publico Jovem Adulto'
amazon_df.loc[amazon_df.genero == 'Action, Drama, Faith and Spirituality',['genero']] = 'Acao, Drama, Fe e Espiritualidade'
amazon_df.loc[amazon_df.genero == 'Drama, Music Videos and Concerts',['genero']] = 'Drama, Musica Videos e Concertos'
amazon_df.loc[amazon_df.genero == 'Fantasy, Suspense',['genero']] = 'Fantasia, Suspense'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Fantasy, Kids',['genero']] = 'Artes, Entretenimento e Cultura, Fantasia, Infantil'
amazon_df.loc[amazon_df.genero == 'Comedy, International, LGBTQ',['genero']] = 'Comedia, Internacional, LGBTQ'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Fantasy, Horror',['genero']] = 'Artes, Entretenimento e Cultura, Fantasia, Terror'
amazon_df.loc[amazon_df.genero == 'Drama, International, Kids',['genero']] = 'Drama, Internacional, Infantil'

amazon_df.loc[amazon_df.genero == 'Arthouse, Comedy, Documentary',['genero']] = 'Casa de Arte, Comedia, Documentario'
amazon_df.loc[amazon_df.genero == 'Faith and Spirituality, Special Interest, Western',['genero']] = 'Fe e Espiritualidade, Conteudo Especial, Ocidental'
amazon_df.loc[amazon_df.genero == 'Adventure, Suspense',['genero']] = 'Aventur, Suspense'
amazon_df.loc[amazon_df.genero == 'Action, Comedy, Suspense',['genero']] = 'Acao, Comedia, Suspense'
amazon_df.loc[amazon_df.genero == 'Anime, Kids, Science Fiction',['genero']] = 'Anime, Infantil, Ficcao Cientifica'
amazon_df.loc[amazon_df.genero == 'Action, Comedy, Young Adult Audience',['genero']] = 'Acao, Comedia, Publico Jovem Adulto'
amazon_df.loc[amazon_df.genero == 'Animation, Suspense',['genero']] = 'Animacao, Suspense'
amazon_df.loc[amazon_df.genero == 'Drama, Romance, Western',['genero']] = 'Drama, Romance, Ocidental'
amazon_df.loc[amazon_df.genero == 'Adventure, Western',['genero']] = 'Aventura, Ocidental'
amazon_df.loc[amazon_df.genero == 'Horror, International, Suspense',['genero']] = 'Terror, Internacional, Suspense'
amazon_df.loc[amazon_df.genero == 'Adventure, Drama, Romance',['genero']] = 'Aventura, Drama, Romance'
amazon_df.loc[amazon_df.genero == 'Animation, Music Videos and Concerts',['genero']] = 'Animacao, Musica Videos e Concertos'
amazon_df.loc[amazon_df.genero == 'Comedy, Drama, Faith and Spirituality',['genero']] = 'Comedia, Drama, Fe e Espiritualidade'
amazon_df.loc[amazon_df.genero == 'Comedy, Kids, LGBTQ',['genero']] = 'Comedia, Infantil, LGBTQ'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, LGBTQ, Romance',['genero']] = 'Artes, Entretenimento e Cultura, LGBTQ, Romance'
amazon_df.loc[amazon_df.genero == 'Special Interest, Suspense',['genero']] = 'Conteudo Especial, Suspense'



In [36]:
amazon_df.loc[amazon_df.genero == 'Horror, Romance, Science Fiction',['genero']] = 'Terror, Romance, Ficca Cientifica'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Fantasy, Science Fiction',['genero']] = 'Artes, Entretenimento e Cultura, Fantasia, Ficcao Cientifica'
amazon_df.loc[amazon_df.genero == 'Drama, Military and War, Suspense',['genero']] = 'Drama, Militar e Guerra, Suspense'
amazon_df.loc[amazon_df.genero == 'Fantasy, Horror',['genero']] = 'Fantasia, Terror'
amazon_df.loc[amazon_df.genero == 'Animation, Science Fiction',['genero']] = 'Animacao, Ficcao Cientifica'
amazon_df.loc[amazon_df.genero == 'Animation, Drama, Science Fiction',['genero']] = 'Animacao, Drama, Ficcao Cientifica'
amazon_df.loc[amazon_df.genero == 'Faith and Spirituality, Music Videos and Concerts',['genero']] = 'Fe e Espiritualidade, Musica Videos e Concertos'
amazon_df.loc[amazon_df.genero == 'Documentary, Faith and Spirituality, Music Videos and Concerts',['genero']] = 'Documentario, Fe e Espiritualidade, Musica Videos e Concertos'
amazon_df.loc[amazon_df.genero == 'Arthouse, Drama, LGBTQ',['genero']] = 'Casa de Arte, Drama, LGBTQ'
amazon_df.loc[amazon_df.genero == 'Comedy, Military and War, Suspense',['genero']] = 'Comedia, Militar e Guerra, Suspense'
amazon_df.loc[amazon_df.genero == 'Faith and Spirituality, Kids, Special Interest',['genero']] = 'Fe e Espiritualidade, Infatil, Conteudo Especial'
amazon_df.loc[amazon_df.genero == 'Kids, Young Adult Audience',['genero']] = 'Infantil, Publico Jovem Adulto'
amazon_df.loc[amazon_df.genero == 'Animation, Drama, Kids',['genero']] = 'Animacao, Drama, Infantil'
amazon_df.loc[amazon_df.genero == 'Animation, Comedy, Drama',['genero']] = 'Animacao, Comedia, Drama'
amazon_df.loc[amazon_df.genero == 'Comedy, Kids, Science Fiction',['genero']] = 'Comedia, Infantil, Ficcao Cientifica'
amazon_df.loc[amazon_df.genero == 'Action, Adventure, Faith and Spirituality',['genero']] = 'Acao, Aventura, Fe e Espiritualidade'
amazon_df.loc[amazon_df.genero == 'Drama, Suspense, Western',['genero']] = 'Drama, Suspense, Ocidental'
amazon_df.loc[amazon_df.genero == 'Arthouse, LGBTQ, Romance',['genero']] = 'Casa de Arte, LGBTQ, Romance'
amazon_df.loc[amazon_df.genero == 'Action, Adventure, Sports',['genero']] = 'Acao, Aventura, Esportes'
amazon_df.loc[amazon_df.genero == 'Comedy, Drama, Military and War',['genero']] = 'Comedia, Drama, Militar e Guerra'
amazon_df.loc[amazon_df.genero == 'Action, Anime',['genero']] = 'Acao, Anime'


In [37]:
amazon_df.loc[amazon_df.genero == 'Comedy, Documentary',['genero']] = 'Comedia, Documentario'
amazon_df.loc[amazon_df.genero == 'Drama, International, Special Interest',['genero']] = 'Drama, Internacional, Conteudo Especial'
amazon_df.loc[amazon_df.genero == 'Drama, Special Interest, Suspense',['genero']] = 'Drama, Conteudo Especial, Suspense'
amazon_df.loc[amazon_df.genero == 'Action, Arts, Entertainment, and Culture, Music Videos and Concerts',['genero']] = 'Acao, Artes, Entretenimento e Cultura, Musica Videos e Concertos'
amazon_df.loc[amazon_df.genero == 'Drama, Romance, Special Interest',['genero']] = 'Drama, Romance, Conteudo Especial'
amazon_df.loc[amazon_df.genero == 'Action, Documentary, Drama',['genero']] = 'Acao, Documentario, Drama'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Kids',['genero']] = 'Artes, Entretenimento e Cultura, Infantil'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Drama, Science Fiction',['genero']] = 'Artes, Entretenimento e Cultura, Drama, Ficcao Cientifica'
amazon_df.loc[amazon_df.genero == 'Arthouse, LGBTQ, Suspense',['genero']] = 'Casa de Arte, LGBTQ, Suspense'
amazon_df.loc[amazon_df.genero == 'Drama, Horror, Romance',['genero']] = 'Drama, Terror, Romance'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Drama, Music Videos and Concerts',['genero']] = 'Artes, Entretenimento e Cultura, Drama, Musica Videos e Concertos'
amazon_df.loc[amazon_df.genero == 'Action, Arthouse, Romance',['genero']] = 'Acaon, Casa de Arte, Romance'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Kids, Science Fiction',['genero']] = 'Artes, Entretenimento e Cultura, Infantil, Ficcao Cientifica'
amazon_df.loc[amazon_df.genero == 'Arthouse, Documentary, Special Interest',['genero']] = 'Casa de Arte, Documentario, Conteudo Especial'
amazon_df.loc[amazon_df.genero == 'Drama, Horror, International',['genero']] = 'Drama, Terror, Internacional'
amazon_df.loc[amazon_df.genero == 'Horror, International, Science Fiction',['genero']] = 'Terror, Internacional, Ficcao Cientifica'
amazon_df.loc[amazon_df.genero == 'Animation, Young Adult Audience',['genero']] = 'Animacao, Publico Jovem Adulto'
amazon_df.loc[amazon_df.genero == 'Action, Anime, Kids',['genero']] = 'Acao, Anime, Infantil'
amazon_df.loc[amazon_df.genero == 'Drama, Romance, Science Fiction',['genero']] = 'Drama, Romance, Ficcao Cientifica'
amazon_df.loc[amazon_df.genero == 'Animation, Romance',['genero']] = 'Animacao, Romance'
amazon_df.loc[amazon_df.genero == 'Action, Romance, Suspense',['genero']] = 'Acao, Romance, Suspense'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Comedy, Suspense',['genero']] = 'Artes, Entretenimento e Cultura, Comedia, Suspense'
amazon_df.loc[amazon_df.genero == 'Action, Adventure, Horror',['genero']] = 'Acao, Aventura, Terror'
amazon_df.loc[amazon_df.genero == 'Action, Adventure, Special Interest',['genero']] = 'Acao, Aventura, Conteudo Especial'
amazon_df.loc[amazon_df.genero == 'Comedy, Fantasy, Young Adult Audience',['genero']] = 'Comedia, Fantasia, Publico Jovem Adulto'
amazon_df.loc[amazon_df.genero == 'Arthouse, Comedy, LGBTQ',['genero']] = 'Casa de Arte, Comedia, LGBTQ'
amazon_df.loc[amazon_df.genero == 'Arthouse, Comedy, Special Interest',['genero']] = 'Casa de Arte, Comedia, Conteudo Especial'
amazon_df.loc[amazon_df.genero == 'Animation, Kids, Science Fiction',['genero']] = 'Animacao, Infantil, Ficcao Cientifica'
amazon_df.loc[amazon_df.genero == 'Animation, Kids, Young Adult Audience',['genero']] = 'Animacao, Infantil, Publico Jovem Adulto'
amazon_df.loc[amazon_df.genero == 'Comedy, Romance, Young Adult Audience',['genero']] = 'Comedia, Romance, Publico Jovem Adulto'
amazon_df.loc[amazon_df.genero == 'International, LGBTQ, Suspense',['genero']] = 'Internacional, LGBTQ, Suspense'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, LGBTQ',['genero']] = 'Casa de Arte, Comedia, LGBTQ'
amazon_df.loc[amazon_df.genero == 'Drama, Special Interest, Sports',['genero']] = 'Drama, Conteudo Especial, Esportes'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Documentary, International',['genero']] = 'Artes, Entretenimento e Cultura, Documentario, Internacional'
amazon_df.loc[amazon_df.genero == 'Action, Romance, Science Fiction',['genero']] = 'Acao, Romance, Ficca Cientifica'
amazon_df.loc[amazon_df.genero == 'Comedy, Drama, Music Videos and Concerts',['genero']] = 'Comedia, Drama, Musica Videos e Concertos'
amazon_df.loc[amazon_df.genero == 'Comedy, Drama, Western',['genero']] = 'Comedia,Casa de Arte, Publico Jovem Adulto'
amazon_df.loc[amazon_df.genero == 'Action, International, Romance',['genero']] = 'Acao, Internacional, Romance'
amazon_df.loc[amazon_df.genero == 'Arthouse, Horror, Science Fiction',['genero']] = 'Casa de Arte, Terror, Ficcao Cientifica'


amazon_df.loc[amazon_df.genero == 'Action, Arts, Entertainment, and Culture',['genero']] = 'Acao, Artes, Entretenimento e Cultura'
amazon_df.loc[amazon_df.genero == 'Arthouse',['genero']] = 'Casa de Arte'
amazon_df.loc[amazon_df.genero == 'Arthouse, Comedy, Horror',['genero']] = 'Casa de Arte, Comedia, Terror'
amazon_df.loc[amazon_df.genero == 'Documentary, Kids, Science Fiction',['genero']] = 'Documentario, Infantil, Ficcao Cientifica'
amazon_df.loc[amazon_df.genero == 'Drama, Historical, Romance',['genero']] = 'Drama, Historico, Romance'
amazon_df.loc[amazon_df.genero == 'Action, Kids, Young Adult Audience',['genero']] = 'Acao, Infantil, Publico Jovem Adulto'
amazon_df.loc[amazon_df.genero == 'Animation, Horror',['genero']] = 'Animacao, Terror'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Special Interest, Unscripted',['genero']] = 'Artes, Entretenimento e Cultura, Conteudo Especial, Improvisado'
amazon_df.loc[amazon_df.genero == 'Horror, Romance, Western',['genero']] = 'Terror, Romance, Ocidental'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Comedy, Horror',['genero']] = 'Artes, Entretenimento e Cultura, Comedia, Terror'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Young Adult Audience',['genero']] = 'Artes, Entretenimento e Cultura, Conteudo Jovem Adulto'
amazon_df.loc[amazon_df.genero == 'Arthouse, Drama, Horror',['genero']] = 'Casa de Arte, Drama, Terror'
amazon_df.loc[amazon_df.genero == 'Comedy, Drama, Horror',['genero']] = 'Comedy, Drama, Terror'
amazon_df.loc[amazon_df.genero == 'Horror, Romance',['genero']] = 'Terror, Romance'
amazon_df.loc[amazon_df.genero == 'Animation, Kids, Suspense',['genero']] = 'Animacao, Infantil, Suspense'
amazon_df.loc[amazon_df.genero == 'Horror, Romance, Suspense',['genero']] = 'Terror, Romance, Suspense'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Drama, Suspense',['genero']] = 'Artes, Entretenimento e Cultura, Drama, Suspense'

amazon_df.loc[amazon_df.genero == 'Kids, Special Interest, Sports',['genero']] = 'Infantil, Conteudo Especial, Esportes'
amazon_df.loc[amazon_df.genero == 'Action, Animation, Drama',['genero']] = 'Acao, Animacao, Drama'
amazon_df.loc[amazon_df.genero == 'Documentary, Horror, Suspensee',['genero']] = 'Documentario, Terror, Suspense'
amazon_df.loc[amazon_df.genero == 'Action, Animation, Science Fiction',['genero']] = 'Acao, Animacao, Ficcao Cientifica'
amazon_df.loc[amazon_df.genero == 'Comedy, Romance, Special Interest',['genero']] = 'Comedia, Romance, Conteudo Especial'
amazon_df.loc[amazon_df.genero == 'Arthouse, Drama, Science Fiction',['genero']] = 'Casa de Arte, Drama, Ficcao Cientifica'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Drama, LGBTQ',['genero']] = 'Artes, Entretenimento e Cultura, Drama, LGBTQ'
amazon_df.loc[amazon_df.genero == 'Animation, Special Interest',['genero']] = 'Animacao, Conteudo Especial'
amazon_df.loc[amazon_df.genero == 'Romance, Suspense, Western',['genero']] = 'Romance, Suspense, Ocidental'
amazon_df.loc[amazon_df.genero == 'Documentary, Drama, Faith and Spirituality',['genero']] = 'Documentario, Drama, Fe e Espiritualidade'
amazon_df.loc[amazon_df.genero == 'Drama, International, Western',['genero']] = 'Drama, Internacional, Ocidental'
amazon_df.loc[amazon_df.genero == 'Arthouse, Comedy, Science Fiction',['genero']] = 'Casa de Arte, Comedia, Ficcao Cientifica'
amazon_df.loc[amazon_df.genero == 'Comedy, Horror, Science Fiction',['genero']] = 'Comedia, Terror, Ficcao Cientifica'
amazon_df.loc[amazon_df.genero == 'Action, Comedy, Romance',['genero']] = 'Acao, Comedia, Romance'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Comedy, Kids',['genero']] = 'Artes, Entretenimento e Cultura, Comedia, Infantil'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Comedy, Faith and Spirituality',['genero']] = 'Artes, Entretenimento e Cultura, Comedia, Fe e Espiritualidade'
amazon_df.loc[amazon_df.genero == 'Action, Special Interest, Suspense',['genero']] = 'Acao, Conteudo Especial, Suspense'

amazon_df.loc[amazon_df.genero == 'Romance, Science Fiction, Suspense',['genero']] = 'Romance, Ficcao Cientifica, Suspense'
amazon_df.loc[amazon_df.genero == 'Documentary, Suspense, Unscripted',['genero']] = 'Documentario, Suspense, Improvisado'
amazon_df.loc[amazon_df.genero == 'Arthouse, Arts, Entertainment, and Culture',['genero']] = 'Casa de Arte, Artes, Entretenimento e Cultura'
amazon_df.loc[amazon_df.genero == 'Drama, Fantasy, Sports',['genero']] = 'Drama, Fantasia, Esportes'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Drama, Horror, Suspense',['genero']] = 'Artes, Entretenimento e Cultura, Drama, Terror, Suspense'
amazon_df.loc[amazon_df.genero == 'Horror, Western',['genero']] = 'Terror, Ocidental'
amazon_df.loc[amazon_df.genero == 'Romance, Western',['genero']] = 'Romance, Ocidental'
amazon_df.loc[amazon_df.genero == 'Anime, Comedy, Drama',['genero']] = 'Anime, Comedia, Drama'
amazon_df.loc[amazon_df.genero == 'Adventure, Science Fiction, Suspense',['genero']] = 'Aventura, Ficcao Cientifica, Suspense'
amazon_df.loc[amazon_df.genero == 'Arthouse, Comedy, International',['genero']] = 'Casa de Arte, Comedia, Internacional'
amazon_df.loc[amazon_df.genero == 'Adventure, Drama, Fantasy',['genero']] = 'Aventura, Drama, Fantasia'
amazon_df.loc[amazon_df.genero == 'Special Interest, Talk Show and Variety, Unscripted',['genero']] = 'Conteudo Especial, Entrevista e Variedade, Improvisado'
amazon_df.loc[amazon_df.genero == 'Faith and Spirituality, Kids, Western',['genero']] = 'Acao, Conteudo Especial, Suspense'
amazon_df.loc[amazon_df.genero == 'Drama, Kids, Suspense',['genero']] = 'Drama, Infantil, Suspense'
amazon_df.loc[amazon_df.genero == 'Arthouse, Horror, International',['genero']] = 'Casa de Arte, Terror, Internacional'

amazon_df.loc[amazon_df.genero == 'Adventure, Arthouse, Horror',['genero']] = 'Aventura, Casa de Arte, Terror'
amazon_df.loc[amazon_df.genero == 'Comedy, Fantasy',['genero']] = 'Comedia, Fantasia'
amazon_df.loc[amazon_df.genero == 'Arthouse, Horror, Romance',['genero']] = 'Casa de Arte, Artes, Entretenimento e Cultura, Terror'
amazon_df.loc[amazon_df.genero == 'Arthouse, Arts, Entertainment, and Culture, Horror',['genero']] = 'Acao, Conteudo Especial, Suspense'
amazon_df.loc[amazon_df.genero == 'Action, Arts, Entertainment, and Culture, Suspense',['genero']] = 'Acao, Artes, Entretenimento e Cultura, Suspense'
amazon_df.loc[amazon_df.genero == 'Animation, Anime, Romance',['genero']] = 'Animacao, Anime, Romance'
amazon_df.loc[amazon_df.genero == 'Military and War',['genero']] = 'Militar e Guerra'
amazon_df.loc[amazon_df.genero == 'Arthouse, Drama, Kids',['genero']] = 'Casa de Arte, Drama, Infantil'
amazon_df.loc[amazon_df.genero == 'Action, Anime, Drama',['genero']] = 'Acao, Anime, Drama'
amazon_df.loc[amazon_df.genero == 'Adventure, Comedy, Unscripted',['genero']] = 'Acventura, Comedia, Improvisado'
amazon_df.loc[amazon_df.genero == 'Arthouse, Documentary, Music Videos and Concerts',['genero']] = 'Casa de Arte, Documentario, Musica, Videos e Concertos'
amazon_df.loc[amazon_df.genero == 'Arthouse, Romance',['genero']] = 'Casa de Arte, Romance'
amazon_df.loc[amazon_df.genero == 'Anime, Drama, Sports',['genero']] = 'Anime, Drama, Esportes'
amazon_df.loc[amazon_df.genero == 'Comedy, Documentary, Drama',['genero']] = 'Comedia, Documentario, Drama'
amazon_df.loc[amazon_df.genero == 'Drama, Kids, Western',['genero']] = 'Drama, Infantil, Ocidental'
amazon_df.loc[amazon_df.genero == 'Comedy, Horror, Kids',['genero']] = 'Comedia, Terror, Infantil'





In [35]:
amazon_df.loc[amazon_df.genero == 'Action, Drama, Fantasy',['genero']] = 'Acao, Drama, Fantasia'
amazon_df.loc[amazon_df.genero == 'Documentary, Horror, Suspense',['genero']] = 'Documentario, Terror, Suspense'
amazon_df.loc[amazon_df.genero == 'Documentary, LGBTQ, Special Interest',['genero']] = 'Documentario, LGBTQ, Conteudo Especial'
amazon_df.loc[amazon_df.genero == 'Arthouse, Horror',['genero']] = 'Casa de Arte, Terror'
amazon_df.loc[amazon_df.genero == 'Arthouse, Horror, Suspense',['genero']] = 'Casa de Arte, Terror, Suspense'
amazon_df.loc[amazon_df.genero == 'Arthouse, Young Adult Audience',['genero']] = 'Casa de Arte, Publico Jovem Adulto'
amazon_df.loc[amazon_df.genero == 'Arts, Entertainment, and Culture, Drama, Horror',['genero']] = 'Artes, Entretenimento e Cultura, Drama, Terror'
amazon_df.loc[amazon_df.genero == 'Comedy, Drama, Special Interest',['genero']] = 'Comedia, Drama, Conteudo Especial'
amazon_df.loc[amazon_df.genero == 'Comedy, Drama, Terror',['genero']] = 'Comedia, Drama, Terror'
amazon_df.loc[amazon_df.genero == 'Comedy, Faith and Spirituality, Kids',['genero']] = 'Comedia, Fe e Espiritualidade, Infantil'
amazon_df.loc[amazon_df.genero == 'Comedy, Horror, Romance',['genero']] = 'Comedia, Terror, Romance'
amazon_df.loc[amazon_df.genero == 'Comedy, Unscripted',['genero']] = 'Comedia, Improvisado'
amazon_df.loc[amazon_df.genero == 'Drama, Western',['genero']] = 'Drama, Ocidental'



In [ ]:
pd.unique(amazon_df.genero)

In [ ]:
pd.unique(amazon_df.duracao)

In [ ]:
df = amazon_df[['genero']]
df.head(80)

In [ ]:
df = amazon_df[['titulo','classificacao']].groupby(amazon_df.classificacao)
df.head(10)

In [ ]:
amazon_df.groupby('classificacao').sum().head(10)

In [38]:
amazon_df.head()

,Unnamed: 0,id,tipo,titulo,diretor,elenco,pais,data_adicionado,ano_lancamento,classificacao,duracao,genero
0,0,s1,Filme,A Grande Sedução,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,2021-03-30,2014,Não avaliado,113 min,"Comedia, Drama"
1,1,s2,Filme,Cuide da boa noite,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,2021-03-30,2018,13+,110 min,"Drama, Internacional"
2,2,s3,Filme,Segredos da Decepção,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",Estados Unidos,2021-03-30,2017,Não avaliado,74 min,"Acao, Drama, Suspense"
3,3,s4,Filme,Rosa: Mantendo-se Verdadeiro,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",Estados Unidos,2021-03-30,2014,Não avaliado,69 min,Documentario
4,4,s5,Filme,Criador de Monstros,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",Reino Unido,2021-03-30,1989,Não avaliado,45 min,"Drama, Fantasia"


Salvando em arquivo.csv



In [39]:
amazon_df.to_csv('Amazon2021_tratado3.csv', index=False)



**Iniciando com PySpark**
>------------------------------------------------------
